In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from dateutil.parser import parse
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.neural_network import MLPRegressor
from imblearn.combine import SMOTEENN
from imblearn.over_sampling import ADASYN
from collections import Counter
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Sequential, load_model, save_model, Model
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import AUC
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf


In [2]:
from xgboost import XGBClassifier

In [3]:
feature_set =  'feature_set_normalised'
X_train = pd.read_csv( F'./data/{feature_set}/X_train_full.csv')
y_train = pd.read_csv( F'./data/{feature_set}/y_train.csv').values.ravel()

X_val = pd.read_csv( F'./data/{feature_set}/X_valid_full.csv')
y_val = pd.read_csv( F'./data/{feature_set}/y_valid.csv').values.ravel()

In [4]:
pos = Counter(y_train).get(0)
neg = Counter(y_train).get(1)
total = neg+pos

In [5]:
weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 19.35
Weight for class 1: 0.51


In [12]:
main_model = XGBClassifier(n_estimators=1000,class_weight=class_weight, eval_metric="auc", early_stopping_rounds=30)

main_model.fit(X_train, y_train, eval_set=[(X_val,y_val)], verbose=True)

[0]	validation_0-auc:0.60439


c:\Users\joshu\anaconda3\lib\site-packages\xgboost\core.py:160: UserWarning: [23:28:24] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "class_weight" } are not used.

  warnings.warn(smsg, UserWarning)


[1]	validation_0-auc:0.61113
[2]	validation_0-auc:0.62124
[3]	validation_0-auc:0.62196
[4]	validation_0-auc:0.62454
[5]	validation_0-auc:0.62391
[6]	validation_0-auc:0.62505
[7]	validation_0-auc:0.63519
[8]	validation_0-auc:0.63555
[9]	validation_0-auc:0.63853
[10]	validation_0-auc:0.64719
[11]	validation_0-auc:0.65403
[12]	validation_0-auc:0.65548
[13]	validation_0-auc:0.65752
[14]	validation_0-auc:0.66942
[15]	validation_0-auc:0.67148
[16]	validation_0-auc:0.67242
[17]	validation_0-auc:0.67157
[18]	validation_0-auc:0.67243
[19]	validation_0-auc:0.67299
[20]	validation_0-auc:0.67256
[21]	validation_0-auc:0.68274
[22]	validation_0-auc:0.68749
[23]	validation_0-auc:0.68953
[24]	validation_0-auc:0.69633
[25]	validation_0-auc:0.69695
[26]	validation_0-auc:0.69573
[27]	validation_0-auc:0.69613
[28]	validation_0-auc:0.69616
[29]	validation_0-auc:0.69592
[30]	validation_0-auc:0.69595
[31]	validation_0-auc:0.69659
[32]	validation_0-auc:0.69678
[33]	validation_0-auc:0.69692
[34]	validation_0-a

XGBClassifier(base_score=None, booster=None, callbacks=None,
              class_weight={0: 19.348582240422996, 1: 0.5132635970605707},
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=30,
              enable_categorical=False, eval_metric='auc', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=1000, n_jobs=None,
              num_parallel_tree=None, ...)

In [24]:
main_model.predict_proba(X_val)

array([[0.9346592 , 0.06534081],
       [0.97614616, 0.02385385],
       [0.9842252 , 0.01577477],
       ...,
       [0.98338634, 0.01661366],
       [0.9872231 , 0.01277693],
       [0.86063975, 0.13936023]], dtype=float32)

array([0.9346592 , 0.97614616, 0.9842252 , ..., 0.98338634, 0.9872231 ,
       0.86063975], dtype=float32)

In [27]:
roc_auc_score(y_train,main_model.predict_proba(X_train)[:, 0])

0.21877069892479037